In [1]:
# To verify CUDA
!/usr/local/cuda/bin/nvcc --version
# To Print GPU info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
Sat Apr 10 17:57:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |         

In [2]:
# Define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [3]:
# Mounting Google Drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
# This creates a symbolic link so that the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

ln: failed to create symbolic link '/mydrive/My Drive': File exists


In [6]:
%cd content

/content


In [7]:
!ls

gdrive	sample_data


In [8]:
%cd gdrive

/content/gdrive


In [9]:
%cd MyDrive/

/content/gdrive/MyDrive


In [10]:
!ls

 14.0.4734.1000_ProfessionalPlus_volume_x86_en-us.iso
 best_model.hdf5
'Colab Notebooks'
 Deepfake
 Deepfake_detection_using_deep_learning-master
 Learning-to-See-Moving-Objects-in-the-Dark-master
 Models
'My Drive'
 results
 tensorflow-speech-recognition-challenge
 test
 train
 trainPart1
 videos
 yolov4


In [11]:
%cd trainPart1/
!ls

/content/gdrive/MyDrive/trainPart1
best_model.hdf5  train	val_images


In [12]:
from keras.layers import *
from keras.models import Sequential

In [13]:
import os, shutil
folders = os.listdir("train")

In [14]:
if not os.path.isdir("val_images"):
    os.mkdir("val_images")

In [16]:
SPLIT = 0.9
for f in os.listdir("train"):
    p = os.path.join("val_images",f) 
    if not os.path.isdir(p):
        os.mkdir(p)
    path = "train/"+f
    imgs = os.listdir(path)
    
    split_size = int(SPLIT*len(imgs))
    files_to_move = imgs[split_size:]
    #print(len(files_to_move))
    #print(files_to_move)
    for img_f in files_to_move:
        src = os.path.join(path,img_f)
        dest = os.path.join("val_images/"+f,img_f)
        shutil.move(src,dest)

In [17]:
print("Training Data")
for f in folders:
    path = "train/"+f
    print(f+ " "+str(len(os.listdir(path))))
   

print("\n Validation Data") 
for f in folders:
    path = "val_images/"+f
    print(f+ " "+str(len(os.listdir(path))))
    

Training Data
Sample054 16
Sample053 16
Sample061 16
Sample056 16
Sample062 16
Sample055 16
Sample058 16
Sample060 16
Sample059 16
Sample057 16
Sample050 16
Sample051 16
Sample043 16
Sample049 16
Sample044 16
Sample052 16
Sample045 16
Sample046 16
Sample047 16
Sample048 16
Sample040 16
Sample039 16
Sample042 16
Sample034 16
Sample035 16
Sample036 16
Sample037 16
Sample041 16
Sample038 16
Sample033 16
Sample028 16
Sample025 16
Sample029 16
Sample026 16
Sample032 16
Sample024 16
Sample027 16
Sample023 16
Sample031 16
Sample030 16
Sample013 16
Sample020 16
Sample017 16
Sample015 16
Sample014 16
Sample016 16
Sample021 16
Sample022 16
Sample019 16
Sample018 16
Sample011 16
Sample007 16
Sample008 16
Sample004 16
Sample012 16
Sample010 16
Sample009 16
Sample003 16
Sample006 16
Sample005 16
Sample001 16
Sample002 16

 Validation Data
Sample054 24
Sample053 24
Sample061 24
Sample056 24
Sample062 24
Sample055 24
Sample058 24
Sample060 24
Sample059 24
Sample057 24
Sample050 24
Sample051 24
Sample

In [18]:
from keras.preprocessing.image import ImageDataGenerator


train_gen = ImageDataGenerator(rescale=1.0/2)
val_gen = ImageDataGenerator(rescale=1.0/2)

train_generator = train_gen.flow_from_directory(
    "train/",
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical',
    color_mode='grayscale'
)

val_generator = val_gen.flow_from_directory(
    "val_images/",
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical',
    color_mode='grayscale'
)


Found 992 images belonging to 62 classes.
Found 1488 images belonging to 62 classes.


In [19]:
for x,y in train_generator:
    print(x.shape)
    print(y.shape)
    break

(32, 224, 224, 1)
(32, 62)


In [22]:
# Build a Model 
#New
model = Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(224,224,1)))
model.add(MaxPool2D((2,2)))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))

#model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPool2D((2,2)))





model.add(GlobalAveragePooling2D())



model.add(Flatten())

model.add(Dense(128,activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(64,activation='relu'))
model.add(Dense(62,activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 222, 222, 32)      320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 52, 52, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 128)              

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20, min_delta=0.00001) 
#mc = ModelCheckpoint('best_model.hdf5', monitor='accuracy', verbose=1, save_best_only=True, mode='max')
history = model.fit(
    train_generator,
    epochs = 100,
    steps_per_epoch = 10,
    validation_data = val_generator,
    validation_steps = 5,
    #callbacks=[es,mc]
)

Epoch 1/100
10/10 [==============================] - 12s 832ms/step - loss: 9.2958 - accuracy: 0.0119 - val_loss: 4.5090 - val_accuracy: 0.0188
Epoch 2/100
10/10 [==============================] - 7s 743ms/step - loss: 4.3247 - accuracy: 0.0244 - val_loss: 4.1687 - val_accuracy: 0.0063
Epoch 3/100
10/10 [==============================] - 7s 745ms/step - loss: 4.1521 - accuracy: 0.0255 - val_loss: 4.1413 - val_accuracy: 0.0125
Epoch 4/100
10/10 [==============================] - 7s 762ms/step - loss: 4.1382 - accuracy: 0.0129 - val_loss: 4.1279 - val_accuracy: 0.0125
Epoch 5/100
10/10 [==============================] - 7s 746ms/step - loss: 4.1171 - accuracy: 0.0187 - val_loss: 4.1557 - val_accuracy: 0.0000e+00
Epoch 6/100
10/10 [==============================] - 7s 749ms/step - loss: 4.1275 - accuracy: 0.0186 - val_loss: 4.1272 - val_accuracy: 0.0125
Epoch 7/100
10/10 [==============================] - 7s 753ms/step - loss: 4.1199 - accuracy: 0.0110 - val_loss: 4.1176 - val_accuracy: 0

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss= history.history['val_loss']


In [ ]:
import matplotlib.pyplot as plt
plt.plot(acc,label="accuracy")
plt.plot(val_acc,label="val acc")
#plt.plot(loss,label='loss')
#plt.plot(val_loss,label='val loss')
plt.legend()
plt.show()